In [1]:
import pandas as pd
df = pd.read_csv('zamowienia.csv', sep=';')

In [2]:
import numpy as np

df.iloc[11, 0] = np.nan
df.iloc[112, 0] = np.nan
df.iloc[131, 0] = np.nan
df.loc[60:75, 'Utarg'] = np.nan
df.to_csv('zamowienia_missing.csv', index=False)

In [3]:
!pip install dask[complete] distributed --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 8.2 MB/s eta 0:00:00


In [4]:
import dask.dataframe as dd

ddf = dd.read_csv('zamowienia_missing.csv')
print(ddf.dtypes)

Kraj               string[pyarrow]
Sprzedawca         string[pyarrow]
Data zamowienia    string[pyarrow]
idZamowienia                 int64
Utarg                      float64
dtype: object


In [5]:
ddf['Utarg'].mean().compute()

1552.9661558109833

In [6]:
ddf = dd.read_csv("zamowienia_missing.csv", sample=10000)
ddf['Utarg'].mean().compute()

1552.9661558109833

In [7]:
ddf_sample = dd.read_csv('zamowienia_missing.csv', sample=256*1024)
print(ddf_sample.dtypes)

Kraj               string[pyarrow]
Sprzedawca         string[pyarrow]
Data zamowienia    string[pyarrow]
idZamowienia                 int64
Utarg                      float64
dtype: object


In [8]:
from dask.distributed import Client
from dask.distributed import LocalCluster

cluster = LocalCluster()
client = Client(cluster)
client

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:33675
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43529'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41171'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:39959', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:39959
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:60120
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:38117', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker co

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 12.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33675,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 12.67 GiB
Comm: tcp://127.0.0.1:38117,Total threads: 1
Dashboard: http://127.0.0.1:40957/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:43529,


In [9]:
client = Client(n_workers=4, memory_limit='8GB')
client

/usr/local/lib/python3.10/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36621 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:38787
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:36621/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43185'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35773'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35323'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44065'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:32951', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:32951
INFO:distributed.core:Starting established conne

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:36621/status,
Dashboard: http://127.0.0.1:36621/status,Workers: 4
Total threads: 4,Total memory: 29.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38787,Workers: 4
Dashboard: http://127.0.0.1:36621/status,Total threads: 4
Started: Just now,Total memory: 29.80 GiB
Comm: tcp://127.0.0.1:32951,Total threads: 1
Dashboard: http://127.0.0.1:43557/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:43185,


In [11]:
import os
import time
filepaths = [f'/content/{f}' for f in os.listdir('/content/') if f.endswith('.parquet')]

start_time = time.time()
ddf = dd.read_parquet(filepaths)
end_time = time.time()

print(f"Czas ładowania danych: {end_time - start_time} s")

Czas ładowania danych: 0.20682239532470703 s


In [12]:
top_users = ddf.nlargest(10, 'likes')
start_time = time.time()
top_users.compute()
end_time = time.time()
print(f"Czas operacji: {end_time - start_time} s")

Czas operacji: 6.923036336898804 s


In [13]:
first_half_2019 = ddf[(ddf['date'] >= '2019-01-01') & (ddf['date'] < '2019-07-01')]
start_time = time.time()
first_half_2019.compute()
end_time = time.time()
print(f"Czas operacji: {end_time - start_time} s")

Czas operacji: 27.49426507949829 s


In [14]:
import numpy as np

df_optimized = ddf.copy()

df_optimized['sid'] = df_optimized['sid'].astype(np.int32)
df_optimized['sid_profile'] = df_optimized['sid_profile'].astype(np.int32)
df_optimized['post_type'] = df_optimized['post_type'].astype(np.int16)
df_optimized['likes'] = df_optimized['likes'].astype(np.int32)
df_optimized['comments'] = df_optimized['comments'].astype(np.int32)
df_optimized['following'] = df_optimized['following'].astype(np.int32)
df_optimized['followers'] = df_optimized['followers'].astype(np.int32)
df_optimized['num_posts'] = df_optimized['num_posts'].astype(np.int16)

df_optimized['lang'] = df_optimized['lang'].astype('category')

In [15]:
top_users = df_optimized.nlargest(10, 'likes')
start_time = time.time()
top_users.compute()
end_time = time.time()
print(f"Czas operacji: {end_time - start_time} s")

Czas operacji: 5.546695947647095 s


In [16]:
first_half_2019 = df_optimized[(df_optimized['date'] >= '2019-01-01') & (df_optimized['date'] < '2019-07-01')]
start_time = time.time()
first_half_2019.compute()
end_time = time.time()
print(f"Czas operacji: {end_time - start_time} s")

Czas operacji: 15.934940576553345 s


In [17]:
import dask.array as da
from dask.array import random

chunk_sizes = [(1_000_000, 1_000_000), (500_000, 500_000), (2000, 2000), (1000, 1000), (500, 500)]
for chunk in chunk_sizes:
    darr = da.random.normal(5, 0.2, size=(20_000, 20_000), chunks=chunk)
    start_time = time.time()
    darr.mean(axis=0).compute()
    end_time = time.time()
    print(f"Liczba chunkow: {chunk}, Czas operacji: {end_time - start_time} s")

Liczba chunkow: (1000000, 1000000), Czas operacji: 19.104210376739502 s
Liczba chunkow: (500000, 500000), Czas operacji: 21.405136823654175 s
Liczba chunkow: (2000, 2000), Czas operacji: 18.348315477371216 s
Liczba chunkow: (1000, 1000), Czas operacji: 23.325117349624634 s
Liczba chunkow: (500, 500), Czas operacji: 40.49262547492981 s
